In [2]:
using DataFrames
using CSV
using Query
using TimeZones
using ProfileView

Gtk-Message: 20:58:33.956: Failed to load module "canberra-gtk-module"
Gtk-Message: 20:58:33.956: Failed to load module "pk-gtk-module"
Gtk-Message: 20:58:33.957: Failed to load module "canberra-gtk-module"
Gtk-Message: 20:58:33.957: Failed to load module "pk-gtk-module"


In [11]:
include("radiative_transfer_model.jl")
include("parameters_hainich.jl");

In [4]:
function read_fluxnet(path)
  df = CSV.File(path)|> 
    DataFrame |>
    @select(:datetime, :SW_IN_F, :SW_DIF, :SW_OUT, :LW_IN_F, :LW_OUT, :TS_F_MDS_1, :TA_F, :zenith) |>
  # just change the name of the columns and in next step give them a corrent value
    @rename(:SW_IN_F => :sw_sky,
            :SW_DIF => :sw_sky_d,
            :LW_IN_F => :lw_sky,
            :TA_F => :t_leaf,       # Temperature of air aproximation for leaf T for now
            :TS_F_MDS_1 => :t_soil, # Temp soil at 2cm depth
            :SW_OUT => :sw_out,
            :LW_OUT => :lw_out   ) |>
    @mutate(sw_sky_b = _.sw_sky - _.sw_sky_d,  # SW_IN_F is the total radiation direct + diffuse
            t_leaf = 273.15 + _.t_leaf,       # Temperature of air aproximation for leaf T for now
            t_soil = 273.15 + _.t_soil ) |> # Temp soil at 2cm depth            
    DataFrame
#     df[!, :sw_sky_b] =  df.sw_sky - df.sw_sky_d
#     df.t_leaf = 273.15 .+ df.t_leaf       # Temperature of air aproximation for leaf T for now
#     df.t_soil = 273.15 .+ df.t_soil
    df.datetime = ZonedDateTime.(df.datetime, "yyyy-mm-dd HH:MM:SSzzzz")
    
    return df
end

read_fluxnet (generic function with 1 method)

In [5]:
df = read_fluxnet("../data/fluxnet_hainich_with_zenith.csv")

,datetime,sw_sky,sw_sky_d,sw_out,lw_sky,lw_out,t_soil
,ZonedDa…,Float64,Float64?,Float64?,Float64,Float64?,Float64
1,2016-01-01T00:30:00+01:00,0.0,0.0,0.0,315.43,319.24,276.15
2,2016-01-01T01:00:00+01:00,0.0,0.0,0.0,316.54,318.77,276.165
3,2016-01-01T01:30:00+01:00,0.0,0.0,0.0,317.94,319.61,276.19
4,2016-01-01T02:00:00+01:00,0.0,0.0,0.0,316.63,319.92,276.22
5,2016-01-01T02:30:00+01:00,0.0,0.0,0.0,318.29,320.42,276.235
6,2016-01-01T03:00:00+01:00,0.0,0.0,0.0,318.01,320.26,276.27
7,2016-01-01T03:30:00+01:00,0.0,0.0,0.0,318.76,320.78,276.29
8,2016-01-01T04:00:00+01:00,0.0,0.0,0.0,318.37,320.74,276.31
9,2016-01-01T04:30:00+01:00,0.0,0.0,0.0,318.18,321.27,276.33


In [12]:
@time out = radiative_transfer_over_input(df, params)

  2.723919 seconds (11.35 M allocations: 646.701 MiB, 4.19% gc time)


52608×18 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  4.40587  …  20.0  1.28676  0.575  0.525053
 0.0  0.0  0.0  0.0  0.0  0.0  4.40587     20.0  1.28676  0.575  0.525053
 0.0  0.0  0.0  0.0  0.0  0.0  4.40587     20.0  1.28676  0.575  0.525053
 0.0  0.0  0.0  0.0  0.0  0.0  4.40587     20.0  1.28676  0.575  0.525053
 0.0  0.0  0.0  0.0  0.0  0.0  4.40587     20.0  1.28676  0.575  0.525053
 0.0  0.0  0.0  0.0  0.0  0.0  4.40587  …  20.0  1.28676  0.575  0.525053
 0.0  0.0  0.0  0.0  0.0  0.0  4.40587     20.0  1.28676  0.575  0.525053
 0.0  0.0  0.0  0.0  0.0  0.0  4.40587     20.0  1.28676  0.575  0.525053
 0.0  0.0  0.0  0.0  0.0  0.0  4.40587     20.0  1.28676  0.575  0.525053
 0.0  0.0  0.0  0.0  0.0  0.0  4.40587     20.0  1.28676  0.575  0.525053
 0.0  0.0  0.0  0.0  0.0  0.0  4.40587  …  20.0  1.28676  0.575  0.525053
 0.0  0.0  0.0  0.0  0.0  0.0  4.40587     20.0  1.28676  0.575  0.525053
 0.0  0.0  0.0  0.0  0.0  0.0  4.40587     20.0  1.28676  0.575  0.525053
 ⋮         

In [6]:
@profview out = radiative_transfer_over_input(df, params)

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1,000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint

In [ ]:
@profview out = radiative_transfer_over_input(df, params)

In [55]:
function update(v)
    v = [1,2,3]
end

update (generic function with 1 method)

In [56]:
z = zeros(100,3)

100×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [61]:
a = @view z[1,:]

3-element view(::Array{Float64,2}, 1, :) with eltype Float64:
 0.0
 0.0
 0.0

In [65]:
a = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [68]:
a = view(z, 1, :)

3-element view(::Array{Float64,2}, 1, :) with eltype Float64:
 0.0
 0.0
 0.0

In [70]:
a = [1.,2.,3.]

3-element Array{Float64,1}:
 1.0
 2.0
 3.0

In [71]:
z

100×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [63]:
for r in z
    update(view(z, 1,:))
end

100×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [80]:
A = [1 2; 3 4]
b = view(A, :, 1)
b[:] = [8, 9]
A

2×2 Array{Int64,2}:
 8  2
 9  4